# MNIST Digit Classification With Augmentation 🧩✍️

This notebook demonstrates a simple but powerful machine learning task: **classifying handwritten digits** using the MNIST dataset.

The **MNIST (Modified National Institute of Standards and Technology)** dataset is a classic benchmark in the field of machine learning and computer vision. It contains **70,000 grayscale images** of handwritten digits from 0 to 9—60,000 for training and 10,000 for testing. Each image is **28x28 pixels**.

In this notebook, we will:
- Load and explore the MNIST dataset
- Preprocess the data for model input
- Data Augmentation
- Grid Search
- Train a classifier 
- Evaluate its performance
- Visualize predictions and misclassifications

This project is enhancement of the previous MINST image classification. Let's get started! 🚀

## LOAD DATA

In [1]:
from sklearn.datasets import fetch_openml
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import clone

from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier

In [2]:
minst = fetch_openml('mnist_784', version=1)
minst.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
X, y = minst['data'], minst['target']

In [ ]:
X.shape

(70000,)

In [5]:
y.shape

(70000,)